In [1]:
!pip install flask



In [2]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler


In [3]:
app = Flask(__name__)

In [4]:
data = pd.read_csv("books.csv")

In [5]:
data

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [6]:
# Remove non-numeric columns
numeric_data = data.select_dtypes(include=['float64', 'int64'])

In [7]:
# Fill missing values with 0
numeric_data.fillna(0, inplace=True)

In [8]:
# Standardize the numeric data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_data)


In [9]:
# Compute user-user similarity matrix
user_similarity_matrix = cosine_similarity(scaled_data)

In [10]:
# Use NearestNeighbors to find k-nearest neighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(scaled_data)  # Fit the model on the data

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
def collaborative_filtering(book_title, similarity_matrix=user_similarity_matrix):
    try:
        # Find the index of the book title
        book_index = data[data['original_title'] == book_title].index[0]
        
        # Replace book_index with appropriate user index or item index
        distances, indices = model_knn.kneighbors(scaled_data[book_index, :].reshape(1, -1), n_neighbors=10)

        # Generate recommendations
        recommendations = []
        for i in range(1, len(indices.flatten())):
            recommended_book_index = data.index[indices.flatten()[i]]
            recommended_book_title = data.loc[recommended_book_index, 'original_title']
            recommendations.append(recommended_book_title)
        return recommendations
    except IndexError:
        return []

In [12]:
@app.route('/recommend', methods=['POST'])
def get_recommendations():
    req_data = request.get_json()
    book_title = req_data['book_title']
    recommendations = collaborative_filtering(book_title)
    return jsonify({'recommendations': recommendations})

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Jun/2024 21:52:21] "POST /recommend HTTP/1.1" 200 -


In [ ]:
# # Define a sample user_id for testing
# sample_user_id = 0  # Change this to the desired user ID

In [ ]:
# # Generate recommendations for the sample user_id
# recommendations = collaborative_filtering(sample_user_id)

In [ ]:
# # Print the recommendations
# print("Recommendations for User", sample_user_id, ":", recommendations)